In [1]:
!pip install torch torchvision torchaudio

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms

In [3]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 14 * 14, 128)
        self.fc2 = nn.Linear(128, 10)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.view(-1, 64 * 14 * 14)
        x = F.relu(self.fc1(x))
        return self.fc2(x)

In [4]:
model = CNN()

In [5]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [6]:
criterion = nn.CrossEntropyLoss()

In [7]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081))])

In [8]:
train_dataset = datasets.MNIST(root="./data", train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root="./data", train=False, download=True, transform=transform)

In [9]:
from torch.utils.data import DataLoader

In [10]:
train_dataset

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.1307,), std=0.3081)
           )

In [11]:
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)

In [12]:
test_dataloader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

In [13]:
def train(model, loader, optimizer, criterion, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(loader):
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        if batch_idx % 100 == 0:
            print(f"Train Epoch: {epoch} Loss: {loss.item():4f}")

In [16]:
for epoch in range(1, 6):
    train(model, train_dataloader, optimizer, criterion, epoch)

Train Epoch: 1 Loss: 2.313909
Train Epoch: 1 Loss: 0.113666
Train Epoch: 1 Loss: 0.184654
Train Epoch: 1 Loss: 0.209756
Train Epoch: 1 Loss: 0.190709
Train Epoch: 1 Loss: 0.032210
Train Epoch: 1 Loss: 0.123374
Train Epoch: 1 Loss: 0.015449
Train Epoch: 1 Loss: 0.068340
Train Epoch: 1 Loss: 0.075541
Train Epoch: 2 Loss: 0.134082
Train Epoch: 2 Loss: 0.007074
Train Epoch: 2 Loss: 0.093768
Train Epoch: 2 Loss: 0.011644
Train Epoch: 2 Loss: 0.043047
Train Epoch: 2 Loss: 0.010323
Train Epoch: 2 Loss: 0.046614
Train Epoch: 2 Loss: 0.008021
Train Epoch: 2 Loss: 0.137353
Train Epoch: 2 Loss: 0.014587
Train Epoch: 3 Loss: 0.006181
Train Epoch: 3 Loss: 0.007107
Train Epoch: 3 Loss: 0.008475
Train Epoch: 3 Loss: 0.001624
Train Epoch: 3 Loss: 0.002208
Train Epoch: 3 Loss: 0.010937
Train Epoch: 3 Loss: 0.001331
Train Epoch: 3 Loss: 0.019234
Train Epoch: 3 Loss: 0.009694
Train Epoch: 3 Loss: 0.027032
Train Epoch: 4 Loss: 0.000777
Train Epoch: 4 Loss: 0.008377
Train Epoch: 4 Loss: 0.007733
Train Epoc

In [17]:
def test(model, loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in loader:
            output = model(data)
            pred = output.argmax(dim=1)
            correct += (pred == target).sum().item()
            total += target.size(0)

    acc = 100. * correct / total
    print(f"Test accuracy: {acc:.2f}%")

In [18]:
test(model, test_dataloader)

Test accuracy: 98.78%


In [14]:
saver = []

for(data, target) in train_dataloader:
    saver = data
    print(target)

tensor([9, 3, 6, 6, 4, 8, 8, 5, 4, 8, 9, 8, 5, 8, 9, 6, 1, 0, 4, 8, 0, 9, 0, 7,
        1, 6, 0, 9, 4, 8, 4, 4, 5, 0, 4, 8, 7, 5, 5, 7, 3, 5, 7, 7, 9, 8, 0, 1,
        6, 0, 4, 1, 1, 6, 1, 7, 4, 4, 5, 5, 1, 4, 0, 7])
tensor([5, 6, 6, 4, 2, 7, 4, 7, 4, 7, 6, 0, 3, 7, 4, 4, 1, 8, 9, 7, 1, 1, 1, 4,
        3, 9, 0, 7, 9, 2, 0, 6, 6, 1, 5, 8, 5, 1, 0, 2, 0, 5, 3, 3, 1, 6, 4, 9,
        9, 8, 6, 4, 1, 7, 8, 0, 4, 9, 5, 7, 6, 2, 1, 5])
tensor([3, 9, 3, 5, 4, 5, 1, 8, 3, 6, 2, 6, 4, 1, 0, 7, 6, 2, 1, 0, 5, 3, 7, 1,
        0, 8, 0, 9, 5, 0, 0, 4, 2, 0, 8, 7, 7, 8, 6, 7, 2, 1, 3, 4, 1, 7, 5, 9,
        5, 6, 0, 2, 1, 0, 7, 4, 2, 4, 9, 8, 9, 9, 1, 5])
tensor([8, 6, 6, 7, 2, 7, 3, 9, 4, 2, 1, 6, 8, 4, 3, 7, 6, 9, 8, 1, 1, 5, 9, 3,
        6, 3, 2, 9, 3, 7, 1, 4, 9, 0, 0, 9, 1, 7, 3, 2, 7, 6, 2, 5, 7, 2, 7, 0,
        1, 2, 3, 9, 8, 1, 4, 9, 4, 8, 4, 0, 1, 3, 9, 2])
tensor([8, 4, 0, 7, 4, 0, 6, 5, 1, 8, 3, 8, 0, 3, 4, 4, 9, 9, 3, 5, 0, 9, 3, 9,
        8, 2, 7, 0, 7, 8, 9, 5, 1, 9, 0, 0, 3, 6, 0,

In [15]:
saver[0]

tensor([[[-0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242,
          -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242,
          -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242,
          -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242],
         [-0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242,
          -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242,
          -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242,
          -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242],
         [-0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242,
          -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242,
          -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242,
          -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242],
         [-0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242,
          -0.4242, -0.4242, -0.4242, -0.4242, -0